In [8]:
#%pip install linkedin-jobs-scraper

#Documentacion: https://pypi.org/project/linkedin-jobs-scraper/

In [9]:
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, RemoteFilters

# Change root logger level (default is WARN)
logging.basicConfig(level = logging.INFO)

# Fired once for each successfully processed job
def on_data(data: EventData):
    print('[ON_DATA]', data.title, data.company, data.company_link, data.date, data.link, data.insights, len(data.description))

# Fired once for each page (25 jobs)
def on_metrics(metrics: EventMetrics):
  print('[ON_METRICS]', str(metrics))

def on_error(error):
    print('[ON_ERROR]', error)

def on_end():
    print('[ON_END]')


scraper = LinkedinScraper(
    chrome_executable_path=None, # Custom Chrome executable path (e.g. /foo/bar/bin/chromedriver) 
    chrome_options=None,  # Custom Chrome options here
    headless=True,  # Overrides headless mode only if chrome_options is None
    max_workers=1,  # How many threads will be spawned to run queries concurrently (one Chrome driver for each thread)
    slow_mo=0.5,  # Slow down the scraper to avoid 'Too many requests 429' errors (in seconds)
    page_load_timeout=20  # Page load timeout (in seconds)    
)

# Add event listeners
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)

queries = [
    Query(
        options=QueryOptions(            
            limit=27  # Limit the number of jobs to scrape.            
        )
    ),
    Query(
        query='Engineer',
        options=QueryOptions(
            locations=['United States', 'Europe'],            
            apply_link = True,  # Try to extract apply link (easy applies are skipped). Default to False.
            limit=5,
            filters=QueryFilters(              
                company_jobs_url='https://www.linkedin.com/jobs/search/?f_C=1441%2C17876832%2C791962%2C2374003%2C18950635%2C16140%2C10440912&geoId=92000000',  # Filter by companies.
                relevance=RelevanceFilters.RECENT,
                time=TimeFilters.MONTH,
                type=[TypeFilters.FULL_TIME, TypeFilters.INTERNSHIP],
                experience=None,                
            )
        )
    ),
]

scraper.run(queries)

INFO:li:scraper:('Using strategy AnonymousStrategy',)
INFO:li:scraper:('Starting new query', "Query(query= options=QueryOptions(limit=27 locations=['Worldwide'] optimize=False apply_link=False))")
INFO:li:scraper:('Chrome debugger url', 'http://localhost:54961')
INFO:li:scraper:('[][Worldwide]', 'Opening https://www.linkedin.com/jobs/search?location=Worldwide')
INFO:li:scraper:('[][Worldwide]', 'Trying first selectors set')
INFO:li:scraper:('[][Worldwide]', 'Trying second selectors set')
INFO:li:scraper:('[][Worldwide]', 'OK')
INFO:li:scraper:('[][Worldwide]', 'Starting pagination loop')
INFO:li:scraper:('[][Worldwide]', 'Found 25 jobs')
INFO:li:scraper:('[][Worldwide][1]', 'Processed')


[ON_DATA] Operador de Loja - São Paulo La Guapa Empório e Empanadas  2022-08-23 https://br.linkedin.com/jobs/view/operador-de-loja-s%C3%A3o-paulo-at-la-guapa-emp%C3%B3rio-e-empanadas-3274656083?refId=GtXkc%2FDTtTcMaodKCJPPGw%3D%3D&trackingId=qmRgAmwtiKEDL9GYz8HiRg%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card [] 1197


INFO:li:scraper:('[][Worldwide][2]', 'Processed')


[ON_DATA] Trabajador/a social Asociación Provivienda  2022-09-06 https://bo.linkedin.com/jobs/view/trabajador-a-social-at-asociaci%C3%B3n-provivienda-3274654209?refId=GtXkc%2FDTtTcMaodKCJPPGw%3D%3D&trackingId=E9hkmSb54%2FS5pG7jtRTsgA%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card [] 4362


INFO:li:scraper:('[][Worldwide][3]', 'Processed')


...

[ON_DATA] Product Design Engineer, Pixel Google  2022-09-11 https://www.linkedin.com/jobs/view/product-design-engineer-pixel-at-google-3119917087?refId=yDJQl7ErD7T2DT1xn4xAYw%3D%3D&trackingId=bwAP91wVMD%2Bi3zjxaZAcwg%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card [] 3571


ERROR:li:scraper:('[Engineer][United States][5]', 'Timeout on loading job details')
NoneType: None
ERROR:li:scraper:('[Engineer][United States][5]', 'Timeout on loading job details')
NoneType: None
INFO:li:scraper:('[Engineer][United States][5]', 'Processed')


[ON_DATA] Data Center Mechanical Engineer Google  2022-09-13 https://www.linkedin.com/jobs/view/data-center-mechanical-engineer-at-google-3264927015?refId=yDJQl7ErD7T2DT1xn4xAYw%3D%3D&trackingId=Arc%2BA8p6Xknc8538zc%2BWvg%3D%3D&position=7&pageNum=0&trk=public_jobs_jserp-result_search-card [] 3542


INFO:li:scraper:('Chrome debugger url', 'http://localhost:55169')
INFO:li:scraper:('[Engineer][Europe]', 'Opening https://www.linkedin.com/jobs/search?keywords=Engineer&location=Europe&f_C=1441%2C17876832%2C791962%2C2374003%2C18950635%2C16140%2C10440912&sortBy=DD&f_TPR=r2592000&f_JT=F%2CI&start=0')
INFO:li:scraper:('[Engineer][Europe]', 'Trying first selectors set')
INFO:li:scraper:('[Engineer][Europe]', 'Trying second selectors set')
INFO:li:scraper:('[Engineer][Europe]', 'Failed to load container selector, skip')


[ON_END]


In [10]:
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, RemoteFilters


query = Query(
    query='Engineer',
    options=QueryOptions(
        locations=['United States'],
        optimize=False,
        limit=5,
        filters=QueryFilters(            
            relevance=RelevanceFilters.RECENT,
            time=TimeFilters.MONTH,
            type=[TypeFilters.FULL_TIME, TypeFilters.INTERNSHIP],
            experience=[ExperienceLevelFilters.INTERNSHIP, ExperienceLevelFilters.MID_SENIOR],
            remote=RemoteFilters.REMOTE, # supported only with authenticated session
        )
    )
)

In [11]:
query = Query(    
    options=QueryOptions(        
        filters=QueryFilters(
            # Paste link below
            company_jobs_url='https://www.linkedin.com/jobs/search/?f_C=1441%2C17876832%2C791962%2C2374003%2C18950635%2C16140%2C10440912&geoId=92000000',        
        )
    )
)

In [12]:
import logging

# Change root logger level (default is WARN)
logging.basicConfig(level = logging.DEBUG)

# Change package logger level
logging.getLogger('li:scraper').setLevel(logging.DEBUG)

# Optional: change level to other loggers
logging.getLogger('urllib3').setLevel(logging.WARN)
logging.getLogger('selenium').setLevel(logging.WARN)